<a href="https://colab.research.google.com/github/Isaivargas/neuronalNetwork/blob/master/FashionMNIST_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
import torch
from torchvision import datasets,transforms
from torch import nn, optim
import torch.nn.functional as F

import helper



transform = transforms.Compose([transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = trainloader


class Classifier(nn.Module):
      def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear (784,256)
        self.fc2 = nn.Linear (256,128)
        self.fc3 = nn.Linear (128,64)
        self.fc4 = nn.Linear (64,10)

        self.dropout = nn.Dropout(p=0.2)

      def forward (self,x):

        x =x.view(x.shape[0],-1)
        
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.dropout(F.relu(self.fc2(x)))
        x = self.dropout(F.relu(self.fc3(x)))

        x = F.log_softmax(self.fc4(x),dim=1)

        return x

model     = Classifier()
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(),lr=0.003)

epochs =  10
steps = 0

train_losses,test_losses =[],[]

for e in range(epochs):
  running_loss = 0
  for images,labels in trainloader:

      images = images.view(images.shape[0],-1)
      logps = model(images) 
      loss  =  criterion(logps,labels)

      optimizer.zero_grad()

      loss.backward()
      optimizer.step()

      running_loss += loss.item()
  else:
      test_loss=0
      accuracy =0

      with torch.no_grad():
        model.eval()
        for images ,labels in testloader:
          log_ps = model(images)
          test_loss += criterion(log_ps,labels)
                                 
          ps = torch.exp(log_ps)
          top_p,top_class = ps.topk(1,dim=1)
          equals= top_class == labels.view(*top_class.shape)
          accuracy +=torch.mean(equals.type(torch.FloatTensor))

      model.train()    

      train_losses.append(running_loss/len(trainloader))
      test_losses.append(test_loss/len(testloader))


      print("Epoch: {}/{}.." .format(e+1,epochs),
            "Trainning_loss:{:.3f}.." .format(running_loss/len(trainloader)),
            "Test_loss:{:.3f}..".format(test_loss/len(testloader)),
            "Test_Acuracy:{:.3f}..".format(accuracy/len(testloader))
            )
      
      torch.save(model.state_dict(),'checkpoint.pth')


Epoch: 1/10.. Trainning_loss:0.610.. Test_loss:0.424.. Test_Acuracy:0.848..
Epoch: 2/10.. Trainning_loss:0.482.. Test_loss:0.387.. Test_Acuracy:0.859..
Epoch: 3/10.. Trainning_loss:0.452.. Test_loss:0.372.. Test_Acuracy:0.865..
Epoch: 4/10.. Trainning_loss:0.433.. Test_loss:0.343.. Test_Acuracy:0.875..
Epoch: 5/10.. Trainning_loss:0.417.. Test_loss:0.339.. Test_Acuracy:0.875..
Epoch: 6/10.. Trainning_loss:0.414.. Test_loss:0.325.. Test_Acuracy:0.881..
Epoch: 7/10.. Trainning_loss:0.402.. Test_loss:0.322.. Test_Acuracy:0.883..
Epoch: 8/10.. Trainning_loss:0.397.. Test_loss:0.306.. Test_Acuracy:0.890..
Epoch: 9/10.. Trainning_loss:0.394.. Test_loss:0.319.. Test_Acuracy:0.883..
Epoch: 10/10.. Trainning_loss:0.390.. Test_loss:0.300.. Test_Acuracy:0.890..
